In [ ]:
import re #for regular expressions to allow sorting the frames

def numberRegex(e):
  #Finding all digits in the filename
  x = re.findall("(\d)", e)

  #if there are two digits in filename
  if(len(x)==2):
      return int(x[0]+""+x[1])
  #if there is one digit in the filename
  else:
      return int(x[0])


In [15]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd

#Lucas directory
#C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Ian directory
#H:/Documents/Github repositories/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Directory containing the RF scan data
dataFolder = 'H:/Documents/Github repositories/SE4450-project-code/Preprocessing/P82-W0-S2.mat'
#vecRFDataNames = os.listdir(os.path.join(dataFolder)) #vector with the names of the images in the above folder
#vecRFDataNames.sort(key=numberRegex) #Sorts the images by frame number

#Loading in the matlab file information
rawMatFile = loadmat(dataFolder)

#Contains just the RF data pulled from the .mat file
RFData = [[element for element in upperElement] for upperElement in rawMatFile['rf1']]

zeroArray = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(256):
    if i>10 and i<246:
        for j in range(2592):
            if j>300 and j<2292:
                RFData[j][i] = zeroArray

# Updates inputted matlab file to contain changes from the for loop above
rawMatFile['rf1'] = RFData

# Saves file while applying changes from the for loop above
# matDic = {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Oct 21 17:26:33 2020','__version__':'1.0', '__globals__':'','rf1':RFData}
# savemat("transformTest.mat", matDic, long_field_names=True)

# Saves the mat file without any changes
savemat("transformTestUnchanged.mat", rawMatFile)

#Converting python dictionary into pandas dataframe for easy manipulation (unsure if needed)
RFDataframe = pd.DataFrame(RFData)

print(RFDataframe.head(5))

                                                 0    \
0  [1, 10, 44, 65, 25, -8, -78, -33, 27, -46, -88...   
1  [0, 0, 31, 62, 22, 11, -55, -55, -4, 32, -115,...   
2  [19, 2, 14, 41, 21, 15, -23, -52, -27, 117, -6...   
3  [-86, 0, 2, 32, 36, 28, 0, -30, -52, 131, 1, -...   
4  [-119, 22, 3, 23, 31, 25, 17, -28, -47, 110, 9...   

                                                 1    \
0  [50, -11, -60, 12, 97, 86, -59, -49, -12, -62,...   
1  [27, 0, -60, -40, 95, 143, -7, -81, -22, -58, ...   
2  [-121, 12, -22, -86, 49, 131, 76, -59, -46, -3...   
3  [-95, 14, -6, -72, -10, 61, 111, 0, -66, -17, ...   
4  [16, -123, -5, -25, -38, -5, 104, 76, -50, -21...   

                                                 2    \
0  [-32, 86, 124, 3, -90, -76, -22, -5, -19, 83, ...   
1  [-13, 31, 102, 72, -57, -63, -51, -29, -28, 87...   
2  [104, -16, 34, 106, -5, -49, -49, -43, -19, 67...   
3  [63, -29, -33, 103, 66, 14, -51, -77, -5, 27, ...   
4  [-21, 104, -70, 71, 110, 96, -45, -80, -17,

In [ ]:
#This was a long shot to visualize the RF dataframe
import matplotlib.pyplot as plt
import numpy as np

[m,n] = np.shape(RFData[0:10][0:10])

#Colour Map using Matrix
plt.figure()
plt.imshow(RFData, alpha=0.8)
plt.xticks(np.arange(n))
plt.yticks(np.arange(m))
plt.xlabel('Numbers')
plt.ylabel('Value')
plt.title('Color Maps')